In [ ]:
#@title 檢視 GPU
! nvidia-smi

Fri Mar 18 14:38:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#@title 安裝 Avatarify
!cd /content
!git clone https://github.com/alievk/avatarify.git
%cd avatarify
!git clone https://github.com/alievk/first-order-model.git fomm
!pip install face-alignment==1.0.0 msgpack_numpy pyyaml==5.1
!scripts/download_data.sh

Cloning into 'avatarify'...
remote: Enumerating objects: 1471, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 1471 (delta 19), reused 11 (delta 5), pack-reused 1433
Receiving objects: 100% (1471/1471), 5.65 MiB | 23.52 MiB/s, done.
Resolving deltas: 100% (952/952), done.
/content/avatarify
Cloning into 'fomm'...
remote: Enumerating objects: 211, done.
remote: Total 211 (delta 0), reused 0 (delta 0), pack-reused 211
Receiving objects: 100% (211/211), 58.16 MiB | 39.68 MiB/s, done.
Resolving deltas: 100% (108/108), done.
     |████████████████████████████████| 274 kB 46.3 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44092 sha256=32e0f6c3e226d0ece15b12741a2866d0595c2b4afa554d00e60e4116198c6bb3
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing install

In [ ]:
#@title 執行 ngrok
!scripts/get_ngrok.sh

ngrok is not found, installing...
Done!


In [ ]:
#@title 執行 avatarify
%cd /content/avatarify

from subprocess import Popen, PIPE
import shlex
import json
import time


def run_with_pipe(command):
  commands = list(map(shlex.split,command.split("|")))
  ps = Popen(commands[0], stdout=PIPE, stderr=PIPE)
  for command in commands[1:]:
    ps = Popen(command, stdin=ps.stdout, stdout=PIPE, stderr=PIPE)
  return ps.stdout.readlines()


def get_tunnel_adresses():
  info = run_with_pipe("curl http://localhost:4040/api/tunnels")
  assert info

  info = json.loads(info[0])
  for tunnel in info['tunnels']:
    url = tunnel['public_url']
    port = url.split(':')[-1]
    local_port = tunnel['config']['addr'].split(':')[-1]
    print(f'{url} -> {local_port} [{tunnel["name"]}]')
    if tunnel['name'] == 'input':
      in_addr = url
    elif tunnel['name'] == 'output':
      out_addr = url
    else:
      print(f'unknown tunnel: {tunnel["name"]}')

  return in_addr, out_addr

# Input and output ports for communication
local_in_port = 5557
local_out_port = 5558

/content/avatarify


In [ ]:
#@title 啟用 Worker
with open('/tmp/run.txt', 'w') as f:
  ps = Popen(
      shlex.split(f'./run.sh --is-worker --in-port {local_in_port} --out-port {local_out_port} --no-vcam --no-conda'),
      stdout=f, stderr=f)
  time.sleep(3)

In [ ]:
#@title 檢視 Worker
!ps aux | grep 'python3 afy/cam_fomm.py' | grep -v grep | tee /tmp/ps_run
!if [[ $(cat /tmp/ps_run | wc -l) == "0" ]]; then echo "Worker failed to start"; cat /tmp/run.txt; else echo "Worker started"; fi

root         201 23.1  8.8 4976684 1181272 ?     Sl   14:38   0:02 python3 afy/cam_fomm.py --config fomm/config/vox-adv-256.yaml --checkpoint vox-adv-cpk.pth.tar --virt-cam 9 --relative --adapt_scale --is-worker --in-port 5557 --out-port 5558 --no-stream
root         227  0.0  2.7 4976684 372436 ?      Sl   14:38   0:00 python3 afy/cam_fomm.py --config fomm/config/vox-adv-256.yaml --checkpoint vox-adv-cpk.pth.tar --virt-cam 9 --relative --adapt_scale --is-worker --in-port 5557 --out-port 5558 --no-stream
root         230  0.0  2.7 4976684 370252 ?      S    14:38   0:00 python3 afy/cam_fomm.py --config fomm/config/vox-adv-256.yaml --checkpoint vox-adv-cpk.pth.tar --virt-cam 9 --relative --adapt_scale --is-worker --in-port 5557 --out-port 5558 --no-stream
root         231  0.0  2.8 4976684 372504 ?      Sl   14:38   0:00 python3 afy/cam_fomm.py --config fomm/config/vox-adv-256.yaml --checkpoint vox-adv-cpk.pth.tar --virt-cam 9 --relative --adapt_scale --is-worker --in-port 5557 --out-po

# 開啟 ngrok tunnel

- 註冊並申請 Token - https://dashboard.ngrok.com/auth/your-authtoken

In [ ]:
# Paste your authtoken here in quotes
authtoken = "" #@param {type:"string"}

# Set your region here in quotes
region = "ap"
config =\
f"""
authtoken: {authtoken}
region: {region}
console_ui: False
tunnels:
  input:
    addr: {local_in_port}
    proto: tcp
  output:
    addr: {local_out_port}
    proto: tcp
"""

with open('ngrok.conf', 'w') as f:
  f.write(config)

# (Re)Open tunnel
ps = Popen('./scripts/open_tunnel_ngrok.sh', stdout=PIPE, stderr=PIPE)
time.sleep(3)

# Get tunnel addresses
try:
  in_addr, out_addr = get_tunnel_adresses()
  print("Tunnel opened")
except Exception as e:
  [print(l.decode(), end='') for l in ps.stdout.readlines()]
  print("Something went wrong, reopen the tunnel")

tcp://0.tcp.ap.ngrok.io:18432 -> 5557 [input]
tcp://0.tcp.ap.ngrok.io:14385 -> 5558 [output]
Tunnel opened


##.啟用客戶端

1. 安裝 avatarify 到本地端作業環境 - https://github.com/alievk/avatarify-python
2. 打開終端機 (Terminal) (Windows 啟用 `Anaconda Prompt`)
3. 將路徑切換至 `avatarify` 目錄
4. 複製以下指令並執行

In [ ]:
#@title 啟用客戶端的 avatarify
print('根據作業環境選擇執行程序\n')
print('Mac:')
print(f'./run_mac.sh --is-client --in-addr {in_addr} --out-addr {out_addr}')
print('\nWindows:')
print(f'run_windows.bat --is-client --in-addr {in_addr} --out-addr {out_addr}')
print('\nLinux:')
print(f'./run.sh --is-client --in-addr {in_addr} --out-addr {out_addr}')

根據作業環境選擇執行程序

Mac:
./run_mac.sh --is-client --in-addr tcp://0.tcp.ap.ngrok.io:18432 --out-addr tcp://0.tcp.ap.ngrok.io:14385

Windows:
run_windows.bat --is-client --in-addr tcp://0.tcp.ap.ngrok.io:18432 --out-addr tcp://0.tcp.ap.ngrok.io:14385

Linux:
./run.sh --is-client --in-addr tcp://0.tcp.ap.ngrok.io:18432 --out-addr tcp://0.tcp.ap.ngrok.io:14385


In [ ]:
#@title 檢視 cam_fomm.log (非必要)
!cat ./var/log/cam_fomm.log | head -100

[1647612127.570800] Loading Predictor


In [ ]:
#@title 檢視 recv_worker.log (非必要)
!cat ./var/log/recv_worker.log | tail -100

[1647612133.740142] Receiving on port 5557


In [ ]:
#@title 檢視 predictor_worker.log (非必要)
!cat ./var/log/predictor_worker.log | tail -100

[1647612183.223677] Initialized predictor with: [[], {'config_path': 'fomm/config/vox-adv-256.yaml', 'checkpoint_path': 'vox-adv-cpk.pth.tar', 'relative': True, 'adapt_movement_scale': True, 'enc_downscale': 1}]


In [ ]:
#@title 檢視 send_worker.log (非必要)
!cat ./var/log/send_worker.log | tail -100

[1647612133.749722] Sending on port 5558
